In [1]:
!pip install tensorflow


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('modelo2.h5')

img_path = 'metal.jpg'
img = image.load_img(img_path, target_size=(32, 32))

img_array = image.img_to_array(img)

img_array = np.expand_dims(img_array, axis=0)

img_array /= 255.0

prediccion = model.predict(img_array)

print(prediccion)

categoriaM = np.argmax(prediccion)

print(f'El índice de la categoría predicha es: {categoriaM} con una probabilidad de {prediccion[0][categoriaM]:.2f}')

In [ ]:
!pip install  tensorflow

In [ ]:
!pip install pyserial

In [10]:
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import time
import serial

COM = 'COM13'
BAUD = '115200'
ser = serial.Serial(COM, BAUD)

def millis():
    return int(round(time.time() * 1000))

tiempoInicio = millis()

model = load_model('modelo2.h5')

cap = cv2.VideoCapture(0)

categorias = ["Vidrio", "Carton", "Metal", "Plastico"]

for categoria in categorias:
    if not os.path.exists(categoria):
        os.makedirs(categoria)

umbral_probabilidad = 0.9

contador_imagenes = {categoria: 0 for categoria in categorias}

while True:
    if ser.in_waiting != 0:
        line = ser.readline().decode('utf-8').rstrip()
        print(line)
        if line == "1":
            
            ret, frame = cap.read()

            if not ret:
                break

            img = cv2.resize(frame, (32, 32))
            img_array = np.expand_dims(img, axis=0) 
            img_array = img_array / 255.0 

            if((millis() - tiempoInicio) > 5000):
                tiempoInicio = millis()
                prediccion = model.predict(img_array)
                categoriaM = np.argmax(prediccion)
                probabilidad_maxima = np.max(prediccion)
                
                nombre_categoria = categorias[categoriaM]

                if probabilidad_maxima >= umbral_probabilidad:
                    contador_imagenes[nombre_categoria] += 1
                    nombre_imagen = f"{nombre_categoria}_{contador_imagenes[nombre_categoria]}.png"
                    ruta_imagen = os.path.join(nombre_categoria, nombre_imagen)
                    cv2.imwrite(ruta_imagen, frame)

                ser.write(f'%{categoriaM};\n'.encode())

            cv2.putText(frame, f'Categoria: {categoriaM}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f'Precisión: {probabilidad_maxima}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
            print(f'Predicción: {prediccion}')
            cv2.imshow('NeuroEcoWaste Solutions', frame)
        

    if cv2.waitKey(1) & 0xFF == ord('s'):
        break

cap.release()
cv2.destroyAllWindows()


..Conexión exitosa a ThingsBoard

 Conectado
Mensaje: Rancés - 000444

1
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 579ms/step
Predicción: [[0.8816516  0.00418125 0.00423634 0.10993085]]
Mensaje: Rancés - 000333

1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicción: [[9.9997759e-01 3.0629570e-07 8.9716332e-07 2.1209466e-05]]
Mensaje: Rancés - 000444

1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicción: [[9.9456006e-01 1.3135598e-05 8.1707221e-06 5.4185903e-03]]
